In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../LIB/')
from env import ENV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
X1 = pd.read_csv(ENV.application_train_ori.value)
X2 = pd.read_csv(ENV.application_test_ori.value)
p_X = pd.read_csv(ENV.previous_application_ori.value)
feature_columns = list(X2.columns.values)

In [3]:
X = pd.concat([X1[feature_columns],X2],axis=0,ignore_index=True,sort=False)
# X.reset_index(drop=True,inplace=True)

In [4]:
class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)

In [5]:
def calculate_na(ser):
    return np.sum(ser.isnull())

def view_hist(ser):
    plt.hist(ser, range=(min(ser.values), max(ser.values)))
    
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [6]:
p = scan_nan_portion(X)
p = p.sort_values()
print(p.describe())

count    121.000000
mean       0.244900
std        0.283705
min        0.000000
25%        0.000000
50%        0.002947
75%        0.505913
max        0.697141
dtype: float64


# Export NAN

In [7]:
p.iloc[0:20]

SK_ID_CURR                     0.0
HOUR_APPR_PROCESS_START        0.0
REG_REGION_NOT_WORK_REGION     0.0
LIVE_REGION_NOT_WORK_REGION    0.0
REG_CITY_NOT_LIVE_CITY         0.0
REG_CITY_NOT_WORK_CITY         0.0
LIVE_CITY_NOT_WORK_CITY        0.0
ORGANIZATION_TYPE              0.0
FLAG_DOCUMENT_21               0.0
FLAG_DOCUMENT_20               0.0
FLAG_DOCUMENT_19               0.0
FLAG_DOCUMENT_18               0.0
FLAG_DOCUMENT_17               0.0
FLAG_DOCUMENT_16               0.0
FLAG_DOCUMENT_15               0.0
FLAG_DOCUMENT_14               0.0
FLAG_DOCUMENT_13               0.0
FLAG_DOCUMENT_12               0.0
FLAG_DOCUMENT_11               0.0
FLAG_DOCUMENT_10               0.0
dtype: float64

# Data Clean

## 0 - 20

In [8]:
category_20 = []
for each in p.iloc[0:20].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        category_20.append(each)
        X[each] = X[each].astype('int')
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')

count    356255.000000
mean     278128.000000
std      102842.104413
min      100001.000000
25%      189064.500000
50%      278128.000000
75%      367191.500000
max      456255.000000
Name: SK_ID_CURR, dtype: float64
----
int64
value counts: 356255
NA percentage: 0.0
count    356255.000000
mean         12.055749
std           3.267576
min           0.000000
25%          10.000000
50%          12.000000
75%          14.000000
max          23.000000
Name: HOUR_APPR_PROCESS_START, dtype: float64
----
int64
value counts: 24
NA percentage: 0.0
count    356255.000000
mean          0.051371
std           0.220753
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: REG_REGION_NOT_WORK_REGION, dtype: float64
----
int64
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
value counts: 2
NA percentage: 0.0
count    356255.000000
mean          0.040847
std           0.197936
min           0.000000
25%           0.000000
50%     

In [9]:
category_20.extend(['ORGANIZATION_TYPE','HOUR_APPR_PROCESS_START'])

##### ORGANIZATION_TYPE

In [10]:
oe = ordinal_encoder()
oe.fit(X['ORGANIZATION_TYPE'],NA_VALUE='XNA')
X['ORGANIZATION_TYPE'] = oe.transform(X['ORGANIZATION_TYPE'])

##### HOUR_APPR_PROCESS_START  -- in previous

In [11]:
col = 'HOUR_APPR_PROCESS_START'
oe.fit(pd.concat([p_X[col],X[col]]))
X[col] = oe.transform(X[col])
p_X[col] = oe.transform(p_X[col])

## 20-40

In [12]:
category_40 = []
failed = []
for each in p.iloc[20:40].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        print('###########################################')
        category_40.append(each)
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)
category_40.remove('REGION_RATING_CLIENT')
category_40.remove('REGION_RATING_CLIENT_W_CITY')

count    356255.000000
mean          0.003977
std           0.062942
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_9, dtype: float64
----
int64
###########################################
value counts: 2
NA percentage: 0.0
count    356255.000000
mean          0.082346
std           0.274891
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_8, dtype: float64
----
int64
###########################################
value counts: 2
NA percentage: 0.0
count    356255.000000
mean          0.000171
std           0.013084
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_7, dtype: float64
----
int64
###########################################
value counts: 2
NA percentage: 0.0
count    356255.000000
mean          0.087976
std           0.28

##### WEEKDAY_APPR_PROCESS_START  -- in previous

In [13]:
col = 'WEEKDAY_APPR_PROCESS_START'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_40.append(col)

exist in previous


### REGION_RATING_CLIENT_W_CITY

In [14]:
col = 'REGION_RATING_CLIENT_W_CITY'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_40.append(col)

##### NAME_HOUSING_TYPE

In [15]:
col = 'NAME_HOUSING_TYPE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_40.append(col)

##### NAME_EDUCATION_TYPE

In [16]:
col = 'NAME_EDUCATION_TYPE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_40.append(col)

##### NAME_FAMILY_STATUS

In [17]:
col = 'NAME_FAMILY_STATUS'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_40.append(col)

# 40 - 60

In [18]:
category_60 = []
failed = []
for each in p.iloc[40:60].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_60.append(each)
        print('{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)

count    356255.000000
mean      -3002.071163
std        1517.901735
min       -7197.000000
25%       -4318.000000
50%       -3252.000000
75%       -1717.000000
max           0.000000
Name: DAYS_ID_PUBLISH, dtype: float64
----
int64
value counts: 6224
NA percentage: 0.0
count    356255.000000
mean          0.414316
std           0.720378
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          20.000000
Name: CNT_CHILDREN, dtype: float64
----
int64
value counts: 16
NA percentage: 0.0
count     356255
unique         2
top            Y
freq      246970
Name: FLAG_OWN_REALTY, dtype: object
----
object
{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
FLAG_OWN_REALTY converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
value counts: 2
NA percentage: 0.0
count     356255
unique         2
top            N
freq      235235
Name: FLAG_OWN_CAR, dtype: object
----
object
{} was appended !!!!!!!!!!!!!!!!!!!!

In [19]:
col = 'FLAG_OWN_REALTY'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)

col = 'FLAG_OWN_CAR'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)

col = 'CODE_GENDER'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]),NA_VALUE='XNA')
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)


col = 'NAME_CONTRACT_TYPE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]),NA_VALUE='XNA')
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)




exist in previous


In [20]:
#################### FILLNA
col = 'DAYS_LAST_PHONE_CHANGE'
if p_X.get(col) is not None:
    p_X[col].fillna(-771)
X[col] = X[col].fillna(-771)

#################### FILLNA
col = 'CNT_FAM_MEMBERS'
if p_X.get(col) is not None:
    p_X[col].fillna(2)
X[col] = X[col].fillna(2)

#################### FILLNA
col = 'AMT_ANNUITY'
if p_X.get(col) is not None:
    p_X[col].fillna(25078)
X[col] = X[col].fillna(25078)

#################### FILLNA
col = 'AMT_GOODS_PRICE'
if p_X.get(col) is not None:
    p_X[col].fillna(4.5*10**5)
X[col] = X[col].fillna(4.5*10**5)

#################### FILLNA
col = 'EXT_SOURCE_2'
if p_X.get(col) is not None:
    p_X[col].fillna(0.53)
X[col] = X[col].fillna(0.53)

#################### FILLNA
col = 'DEF_30_CNT_SOCIAL_CIRCLE'
if p_X.get(col) is not None:
    p_X[col].fillna(0)
X[col] = X[col].fillna(0)

# 60-80

In [21]:
category_80 = []
failed = []
for each in p.iloc[60:80].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_80.append(each)
        print('{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)
category_80.remove('DEF_60_CNT_SOCIAL_CIRCLE')
category_80.remove('AMT_REQ_CREDIT_BUREAU_HOUR')
category_80.remove('AMT_REQ_CREDIT_BUREAU_DAY')
category_80.remove('AMT_REQ_CREDIT_BUREAU_WEEK')

count    355205.000000
mean          0.100198
std           0.368259
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          24.000000
Name: DEF_60_CNT_SOCIAL_CIRCLE, dtype: float64
----
float64
DEF_60_CNT_SOCIAL_CIRCLE was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
DEF_60_CNT_SOCIAL_CIRCLE converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
value counts: 9
NA percentage: 0.0029473270550588763
count    355205.000000
mean          1.409468
std           2.577724
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         351.000000
Name: OBS_60_CNT_SOCIAL_CIRCLE, dtype: float64
----
float64
value counts: 35
NA percentage: 0.0029473270550588763
count    355205.000000
mean          1.425729
std           2.599914
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         354.000000
Name: OBS_30_CNT_SOCIAL_CIRCL

In [22]:
col = 'NAME_TYPE_SUITE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)



col = 'OCCUPATION_TYPE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)


col = 'EMERGENCYSTATE_MODE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)

exist in previous


In [23]:
#################### FILLNA
col = 'DEF_60_CNT_SOCIAL_CIRCLE'
if p_X.get(col) is not None:
    p_X[col].fillna(0)
X[col] = X[col].fillna(0)

#################### FILLNA
col = 'OBS_60_CNT_SOCIAL_CIRCLE'
if p_X.get(col) is not None:
    p_X[col].fillna(0)
X[col] = X[col].fillna(2)

#################### FILLNA
col = 'OBS_30_CNT_SOCIAL_CIRCLE'
if p_X.get(col) is not None:
    p_X[col].fillna(25078)
X[col] = X[col].fillna(25078)

#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_MON'
if p_X.get(col) is not None:
    p_X[col].fillna(0.23)
X[col] = X[col].fillna(0.23)

#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_HOUR'
if p_X.get(col) is not None:
    p_X[col].fillna(0.006)
X[col] = X[col].fillna(0.006)

#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_DAY'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0063)
X[col] = X[col].fillna(0.0063)


#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_WEEK'
if p_X.get(col) is not None:
    p_X[col].fillna(0.03)
X[col] = X[col].fillna(0.03)


#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_YEAR'
if p_X.get(col) is not None:
    p_X[col].fillna(1.91)
X[col] = X[col].fillna(1.91)


#################### FILLNA
col = 'AMT_REQ_CREDIT_BUREAU_QRT'
if p_X.get(col) is not None:
    p_X[col].fillna(0.304)
X[col] = X[col].fillna(0.304)

#################### FILLNA
col = 'EXT_SOURCE_3'
if p_X.get(col) is not None:
    p_X[col].fillna(0.5093)
X[col] = X[col].fillna(0.5093)

#################### FILLNA
col = 'TOTALAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.103)
X[col] = X[col].fillna(0.103)


#################### FILLNA
col = 'YEARS_BEGINEXPLUATATION_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.9772)
X[col] = X[col].fillna(0.9772)


#################### FILLNA
col = 'YEARS_BEGINEXPLUATATION_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.9779)
X[col] = X[col].fillna(0.9779)

#################### FILLNA
col = 'YEARS_BEGINEXPLUATATION_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.9779)
X[col] = X[col].fillna(0.9779)

#################### FILLNA
col = 'FLOORSMAX_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.223)
X[col] = X[col].fillna(0.223)


#################### FILLNA
col = 'FLOORSMAX_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.2269)
X[col] = X[col].fillna(0.2269)

#################### FILLNA
col = 'FLOORSMAX_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.2273)
X[col] = X[col].fillna(0.2273)

# 80-100

In [24]:
category_100 = []
failed = []
for each in p.iloc[80:100].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_100.append(each)
        print('{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)

count    178353.000000
mean          0.108089
std           0.111194
min           0.000000
25%           0.045800
50%           0.074900
75%           0.131000
max           1.000000
Name: LIVINGAREA_AVG, dtype: float64
----
float64
value counts: 5272
NA percentage: 0.49936702642769926
count    178353.000000
mean          0.106641
std           0.112555
min           0.000000
25%           0.043100
50%           0.073300
75%           0.125800
max           1.000000
Name: LIVINGAREA_MODE, dtype: float64
----
float64
value counts: 5382
NA percentage: 0.49936702642769926
count    178353.000000
mean          0.109279
std           0.112881
min           0.000000
25%           0.046200
50%           0.075400
75%           0.131200
max           1.000000
Name: LIVINGAREA_MEDI, dtype: float64
----
float64
value counts: 5360
NA percentage: 0.49936702642769926
count             178339
unique                 3
top       block of flats
freq              175162
Name: HOUSETYPE_MODE, dtype: objec

In [25]:
col = 'HOUSETYPE_MODE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)
###############################################

col = 'HOUSETYPE_MODE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)
###############################################


col = 'WALLSMATERIAL_MODE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)
###############################################

In [26]:
#################### FILLNA
col = 'LIVINGAREA_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1081)
X[col] = X[col].fillna(0.1081)

#################### FILLNA
col = 'LIVINGAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1066)
X[col] = X[col].fillna(0.1066)

#################### FILLNA
col = 'LIVINGAREA_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1092)
X[col] = X[col].fillna(0.1092)

#################### FILLNA
col = 'ENTRANCES_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.15)
X[col] = X[col].fillna(0.15)

#################### FILLNA
col = 'ENTRANCES_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.145)
X[col] = X[col].fillna(0.145)

#################### FILLNA
col = 'ENTRANCES_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.149)
X[col] = X[col].fillna(0.149)

#################### FILLNA
col = 'APARTMENTS_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.118)
X[col] = X[col].fillna(0.118)

#################### FILLNA
col = 'APARTMENTS_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1185)
X[col] = X[col].fillna(0.1185)

#################### FILLNA
col = 'APARTMENTS_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1149)
X[col] = X[col].fillna(0.1149)

#################### FILLNA
col = 'ELEVATORS_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0753)
X[col] = X[col].fillna(0.0753)

#################### FILLNA
col = 'ELEVATORS_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0789)
X[col] = X[col].fillna(0.0789)

#################### FILLNA
col = 'ELEVATORS_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0798)
X[col] = X[col].fillna(0.0798)

#################### FILLNA
col = 'EXT_SOURCE_1'
if p_X.get(col) is not None:
    p_X[col].fillna(0.501965)
X[col] = X[col].fillna(0.501965)

#################### FILLNA
col = 'NONLIVINGAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0272)
X[col] = X[col].fillna(0.0272)

#################### FILLNA
col = 'NONLIVINGAREA_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0285)
X[col] = X[col].fillna(0.0285)
#################### FILLNA
col = 'NONLIVINGAREA_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0284)
X[col] = X[col].fillna(0.0284)
#################### FILLNA
col = 'BASEMENTAREA_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0882)
X[col] = X[col].fillna(0.0882)
#################### FILLNA
col = 'BASEMENTAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.08775)
X[col] = X[col].fillna(0.08775)

# 100-120

In [27]:
category_120 = []
failed = []
for each in p.iloc[100:120].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_120.append(each)
        print('{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)

count    148671.000000
mean          0.088673
std           0.082312
min           0.000000
25%           0.044500
50%           0.076500
75%           0.112300
max           1.000000
Name: BASEMENTAREA_AVG, dtype: float64
----
float64
value counts: 3831
NA percentage: 0.5826837518069922
count    145411.000000
mean          0.065092
std           0.081911
min           0.000000
25%           0.016600
50%           0.045900
75%           0.084300
max           1.000000
Name: LANDAREA_MODE, dtype: float64
----
float64
value counts: 3621
NA percentage: 0.591834500568413
count    145411.000000
mean          0.067296
std           0.082267
min           0.000000
25%           0.018800
50%           0.048700
75%           0.087000
max           1.000000
Name: LANDAREA_MEDI, dtype: float64
----
float64
value counts: 3618
NA percentage: 0.591834500568413
count    145411.000000
mean          0.066454
std           0.081287
min           0.000000
25%           0.018700
50%           0.048200
75%

In [28]:
col = 'FONDKAPREMONT_MODE'
X[col].value_counts()

reg oper account         85954
reg oper spec account    14070
not specified             6600
org spec account          6539
Name: FONDKAPREMONT_MODE, dtype: int64

In [29]:
col = 'FONDKAPREMONT_MODE'
if p_X.get(col) is not None:
    oe.fit(pd.concat([p_X[col],X[col]]))
    X[col] = oe.transform(X[col])
    p_X[col] = oe.transform(p_X[col])
    print('exist in previous')
else:
    oe.fit(X[col])
    X[col] = oe.transform(X[col])
category_60.append(col)
###############################################

In [30]:
#################### FILLNA
col = 'BASEMENTAREA_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0886)
X[col] = X[col].fillna(0.0886)

#################### FILLNA
col = 'LANDAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.065)
X[col] = X[col].fillna(0.065)
#################### FILLNA
col = 'LANDAREA_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0673)
X[col] = X[col].fillna(0.0673)
#################### FILLNA
col = 'LANDAREA_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0665)
X[col] = X[col].fillna(0.0665)
#################### FILLNA
col = 'OWN_CAR_AGE'
if p_X.get(col) is not None:
    p_X[col].fillna(12)
X[col] = X[col].fillna(12)
#################### FILLNA
col = 'YEARS_BUILD_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.75)
X[col] = X[col].fillna(0.75)
#################### FILLNA
col = 'YEARS_BUILD_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.76)
X[col] = X[col].fillna(0.76)
#################### FILLNA
col = 'YEARS_BUILD_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.76)
X[col] = X[col].fillna(0.76)
#################### FILLNA
col = 'FLOORSMIN_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.2289)
X[col] = X[col].fillna(0.2289)
#################### FILLNA
col = 'FLOORSMIN_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.2325)
X[col] = X[col].fillna(0.2325)

#################### FILLNA
col = 'FLOORSMIN_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.2328)
X[col] = X[col].fillna(0.2328)
#################### FILLNA
col = 'LIVINGAPARTMENTS_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1015)
X[col] = X[col].fillna(0.1015)

#################### FILLNA
col = 'LIVINGAPARTMENTS_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1063)
X[col] = X[col].fillna(0.1063)
#################### FILLNA
col = 'LIVINGAPARTMENTS_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.1026)
X[col] = X[col].fillna(0.1026)


#################### FILLNA
col = 'NONLIVINGAPARTMENTS_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.00887)
X[col] = X[col].fillna(0.00887)
#################### FILLNA
col = 'NONLIVINGAPARTMENTS_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0087)
X[col] = X[col].fillna(0.0087)
#################### FILLNA
col = 'NONLIVINGAPARTMENTS_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0081)
X[col] = X[col].fillna(0.0081)
#################### FILLNA
col = 'COMMONAREA_MODE'
if p_X.get(col) is not None:
    p_X[col].fillna(0.0429)
X[col] = X[col].fillna(0.0429)
#################### FILLNA
col = 'COMMONAREA_AVG'
if p_X.get(col) is not None:
    p_X[col].fillna(0.045)
X[col] = X[col].fillna(0.045)

# 120 -

In [31]:
category_140 = []
failed = []
for each in p.iloc[120:].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_140.append(each)
        print('{} was appended !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('===========================')
print(failed)

count    107895.000000
mean          0.044994
std           0.077140
min           0.000000
25%           0.007900
50%           0.021000
75%           0.051800
max           1.000000
Name: COMMONAREA_MEDI, dtype: float64
----
float64
value counts: 3289
NA percentage: 0.6971410927565929
[]


In [32]:
col = 'COMMONAREA_MEDI'
if p_X.get(col) is not None:
    p_X[col].fillna(0.045)
X[col] = X[col].fillna(0.045)

# Process P_X

In [33]:
col = 'AMT_ANNUITY'
p_X[col] = p_X[col].fillna(X[col].mean())

col = 'AMT_CREDIT'
p_X[col] = p_X[col].fillna(X[col].mean())

col = 'AMT_GOODS_PRICE'
p_X[col] = p_X[col].fillna(X[col].mean())

# Saving

In [34]:
train = X.iloc[0:len(X1)].copy()
train['TARGET'] = X1['TARGET']
assert len(train) == len(X1)

test = X.iloc[len(X1):]
assert len(test) == len(X2)

In [35]:
train.to_pickle(ENV.application_train_cleaned.value)
test.to_pickle(ENV.application_test_cleaned.value)

In [36]:
p_X.to_pickle(ENV.previous_application_cleaned.value)

In [37]:
np.sum(X.isnull()).sort_values()

SK_ID_CURR                      0
TOTALAREA_MODE                  0
HOUSETYPE_MODE                  0
FONDKAPREMONT_MODE              0
NONLIVINGAREA_MEDI              0
NONLIVINGAPARTMENTS_MEDI        0
LIVINGAREA_MEDI                 0
LIVINGAPARTMENTS_MEDI           0
LANDAREA_MEDI                   0
FLOORSMIN_MEDI                  0
FLOORSMAX_MEDI                  0
ENTRANCES_MEDI                  0
ELEVATORS_MEDI                  0
WALLSMATERIAL_MODE              0
COMMONAREA_MEDI                 0
YEARS_BEGINEXPLUATATION_MEDI    0
BASEMENTAREA_MEDI               0
APARTMENTS_MEDI                 0
NONLIVINGAREA_MODE              0
NONLIVINGAPARTMENTS_MODE        0
LIVINGAREA_MODE                 0
LIVINGAPARTMENTS_MODE           0
LANDAREA_MODE                   0
FLOORSMIN_MODE                  0
FLOORSMAX_MODE                  0
ENTRANCES_MODE                  0
ELEVATORS_MODE                  0
YEARS_BUILD_MEDI                0
EMERGENCYSTATE_MODE             0
OBS_30_CNT_SOC

In [41]:
categori_col = list(set(category_20+ category_40 + category_60+ category_80+ category_100 + category_120 + category_140))

In [42]:
import pickle
pickle.dump(categori_col,open(ENV.clean_categorical_col.value,'wb'))